In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
pd.set_option('display.max_rows', 1000)
%matplotlib inline

In [2]:
files = {
    'AverageKDTree': 'average_kd_tree',
    'MedianKDTree': 'median_kd_tree',
    'CrackingKDTree': 'cracking_kd_tree',
    'CrackingKDTreePerDimension': 'cracking_kd_tree_pd',
    'Quasii': 'quasii',
    'FullScanBitVector':'full_scan_bv',
    'FullScanCandidateList': 'full_scan_cl',
    'ProgressiveIndex': 'progressive_index',
    'ProgressiveIndexCostModel': 'progressive_index_cm'
}
def read_df(alg_name, delta, partition, exp_name, n_rows, n_queries, n_cols, sel): 
    df = pd.read_csv(f"results/{alg_name}-{delta}-{partition}-{exp_name}-{n_rows}-{n_queries}-{n_cols}-{sel}.csv")
    repetitions = df['repetition'].max() + 1
    step = int(len(df.index)/repetitions)
    df_final = df[:step].copy().reset_index()
    for rep in range(1, repetitions):
        df_final += df[step * (rep) : step * (rep + 1)].copy().reset_index()
    
    df_final = df_final/repetitions
    
    if 'index_search_time' not in df_final:
        df_final['index_search_time'] = 0.0
    df_final['query_time'] = df_final['initialization_time'] + df_final['index_search_time'] + df_final['scan_time'] + df_final['adaptation_time']
    df_final['query_time_cumsum'] = df_final['query_time'].cumsum()
    return df_final

# Cumulative Response Time
Cumulative response time using 2 attributes, 10M rows, and 0.1 selectivity

In [10]:
experiment = 'uniform'
n_rows = '10000000'
n_queries = '3000'
n_cols = '8'
sel='0.001'
fig = go.Figure()

def plot_time(fig, df, name):
    fig.add_trace(go.Scatter(y=df['query_time_cumsum'], mode='lines',name=name))

plot_time(fig, read_df(files['AverageKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Average KDTree')

plot_time(fig, read_df(files['MedianKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols,sel), 'MedianKDTree')

plot_time(fig, read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols,sel), 'Adaptive KDTree')

plot_time(fig, read_df(files['CrackingKDTreePerDimension'], '0.0', '1024', experiment, n_rows, n_queries, n_cols,sel), 'Adaptive KDTree Per Predicate')

plot_time(fig, read_df(files['Quasii'], '0.0', '1024', experiment, n_rows, n_queries, n_cols,sel), 'Quasii')

plot_time(fig, read_df(files['ProgressiveIndex'], '0.1', '1024', experiment, n_rows, n_queries, n_cols,sel), 'ProgressiveIndex (Delta=0.1)')
# plot_time(fig, read_df(files['ProgressiveIndex'], '0.2', '1024', experiment, n_rows, n_queries, n_cols,sel), 'ProgressiveIndex (Delta=0.2)')
# plot_time(fig, read_df(files['ProgressiveIndex'], '0.5', '1024', experiment, n_rows, n_queries, n_cols,sel), 'ProgressiveIndex (Delta=0.5)')

plot_time(fig, read_df(files['ProgressiveIndexCostModel'], '0.1', '1024', experiment, n_rows, n_queries, n_cols,sel), 'ProgressiveIndex Cost Model (Delta=0.1)')
# plot_time(fig, read_df(files['ProgressiveIndexAdaptive'], '0.2', '1024', experiment, n_rows, n_queries, n_cols,sel), 'ProgressiveIndexAdaptive (Delta=0.2)')
# plot_time(fig, read_df(files['ProgressiveIndexAdaptive'], '0.5', '1024', experiment, n_rows, n_queries, n_cols,sel), 'ProgressiveIndexAdaptive (Delta=0.5)')


# plot_time(fig, read_df(files['FullScanCandidateList'], '0.0', '0', experiment, n_rows, n_queries, n_cols,sel), 'Full Scan Candidate List')
# plot_time(fig, read_df(files['FullScanBitVector'], '0.0', '0', experiment, n_rows, n_queries, n_cols,sel), 'Full Scan Bitvector')

# plot_time(fig, read_df(experiment, files['FullScan']), 'Full Scan')
# plot_time(fig, read_df(experiment, files['FullScanCandidate']), 'Full Scan Candidate List')
fig.update_layout(title=f'Cumulative response time ({experiment})',
                   xaxis_title='Query',
                   yaxis_title='Time (seconds)')
fig.show()

# First Query Response Time

In [34]:
# experiment = 'power'
# n_rows = '10000000.0'
# n_queries = '3000'
# sel='0.0'

def first_query(values):
    dfs = np.array(values)[:,0]
    names = np.array(values)[:,1]
    fig = go.Figure()
    first_query_time = np.array([x['query_time'][0] for x in dfs])
    fig = go.Figure(data=[
        go.Bar(name='First Query Tim', x=names, y=first_query_time)
    ])
    fig.update_layout(title=f'First Query Response time ({experiment})', yaxis_title='Time (seconds)')
    fig.show()

first_query([
[read_df(files['AverageKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Average KDTree'],
[read_df(files['MedianKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'MedianKDTree'],
[read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree'],
[read_df(files['CrackingKDTreePerDimension'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree Per Predicate'],
[read_df(files['Quasii'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Quasii'],
[read_df(files['ProgressiveIndex'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.1)'],
# [read_df(files['ProgressiveIndex'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.2)'],
# [read_df(files['ProgressiveIndex'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.5)'],
[read_df(files['ProgressiveIndexAdaptive'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.1)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.2)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.5)'],
# [read_df(files['FullScanCandidateList'], '0.0', '0', experiment, n_rows, n_queries, n_cols, sel), 'Full Scan Candidate List'],
])

In [22]:
experiment = 'sequential'
n_rows = '10000000'
n_queries = '3000'
n_cols = '2'
sel='0.01'

def break_down(values):
    dfs = np.array(values)[:,0]
    names = np.array(values)[:,1]
    fig = go.Figure()
    initializations = np.array([x['initialization_time'].sum() for x in dfs])
    adaptation = np.array([x['adaptation_time'].sum() for x in dfs])
    search = np.array([x['index_search_time'].sum() for x in dfs])
    scan = np.array([x['scan_time'].sum() for x in dfs])
    
    fig = go.Figure(data=[
        go.Bar(name='Initialization', x=names, y=initializations),
        go.Bar(name='Adaptation', x=names, y=adaptation),
        go.Bar(name='Index Search', x=names, y=search),
        go.Bar(name='Scan', x=names, y=scan),
    ])
    
    # Change the bar mode
    fig.update_layout(barmode='stack')
    fig.update_layout(title=f'Time Breakdown ({experiment})',
                   yaxis_title='Time (seconds)')
    fig.show()

break_down([
[read_df(files['AverageKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Average KDTree'],
[read_df(files['MedianKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'MedianKDTree'],
[read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree'],
[read_df(files['CrackingKDTreePerDimension'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree Per Predicate'],
[read_df(files['Quasii'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Quasii'],
[read_df(files['ProgressiveIndex'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.1)'],
# [read_df(files['ProgressiveIndex'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.2)'],
# [read_df(files['ProgressiveIndex'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.5)'],
[read_df(files['ProgressiveIndexCostModel'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexCM (Delta=0.1)'],
[read_df(files['ProgressiveIndexCostModel'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexCM (Delta=0.2)'],
# [read_df(files['FullScanCandidateList'], '0.0', '0', experiment, n_rows, n_queries, n_cols, sel), 'Full Scan Candidate List'],
])

In [16]:
# experiment = 'power'
# n_rows = '10000000.0'
# n_queries = '3000'
# sel='0.0'

def convergence(values):
    dfs = np.array(values)[:,0]
    names = np.array(values)[:,1]
    fig = go.Figure()

    convergences = []
    for df in dfs:
        c = [i for i, x in enumerate(df['adaptation_time']) if x < 0.001]
        if(len(c) == 0):
            convergences.append(n_queries)
        else:
            convergences.append(c[0])
    
    fig = go.Figure(data=[
        go.Bar(name='Query Number', x=names, y=convergences),
    ])
    fig.update_layout(title=f'Convergence ({experiment})',
                   yaxis_title='Query number')
    fig.show()

convergence([
[read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree'],
[read_df(files['CrackingKDTreePerDimension'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree Per Predicate'],
[read_df(files['Quasii'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Quasii'],
[read_df(files['ProgressiveIndex'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.1)'],
# [read_df(files['ProgressiveIndex'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.2)'],
# [read_df(files['ProgressiveIndex'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.5)'],
[read_df(files['ProgressiveIndexAdaptive'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.1)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.2)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.5)'],
])

In [18]:
# experiment = 'power'
# n_rows = '10000000.0'
# n_queries = '3000'
# sel='0.0'

def robustness(values):
    dfs = np.array(values)[:,0]
    names = np.array(values)[:,1]
    fig = go.Figure()

    variances = [np.var(df['query_time'][:30]) for df in dfs]
    fig = go.Figure(data=[
        go.Bar(name='Query Number', x=names, y=variances),
    ])
    fig.update_layout(title=f'Robustness ({experiment})',
                   yaxis_title='Variance of first 30 queries')
    fig.show()

robustness([
# [read_df(files['AverageKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Average KDTree'],
# [read_df(files['MedianKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'MedianKDTree'],
[read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree'],
[read_df(files['CrackingKDTreePerDimension'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree Per Predicate'],
[read_df(files['Quasii'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Quasii'],
[read_df(files['ProgressiveIndex'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.1)'],
# [read_df(files['ProgressiveIndex'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.2)'],
# [read_df(files['ProgressiveIndex'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.5)'],
[read_df(files['ProgressiveIndexAdaptive'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.1)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.2)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.5)'],
# [read_df(files['FullScanCandidateList'], '0.0', '0', experiment, n_rows, n_queries, n_cols, sel), 'Full Scan Candidate List'],
])

In [20]:
# experiment = 'power'
# n_rows = '10000000.0'
# n_queries = '3000'
# sel='0.0'

def payoff(values, baseline):
    dfs = np.array(values)[:,0]
    names = np.array(values)[:,1]
    fig = go.Figure()

    payoffs = []
    for df in dfs:
        c = [i for i, x in enumerate(df['query_time_cumsum'] - baseline['query_time_cumsum']) if x < 0]
        if(len(c) == 0):
            payoffs.append(n_queries)
        else:
            payoffs.append(c[0])
    
    fig = go.Figure(data=[
        go.Bar(name='Query Number', x=names, y=payoffs),
    ])
    fig.update_layout(title=f'Payoff ({experiment})',
                   yaxis_title='Query Number')
    fig.show()

payoff([
    [read_df(files['AverageKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Average KDTree'],
[read_df(files['MedianKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'MedianKDTree'],
[read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree'],
[read_df(files['CrackingKDTreePerDimension'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Adaptive KDTree Per Predicate'],
[read_df(files['Quasii'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel), 'Quasii'],
[read_df(files['ProgressiveIndex'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.1)'],
# [read_df(files['ProgressiveIndex'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.2)'],
# [read_df(files['ProgressiveIndex'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndex (Delta=0.5)'],
[read_df(files['ProgressiveIndexAdaptive'], '0.1', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.1)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.2', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.2)'],
# [read_df(files['ProgressiveIndexAdaptive'], '0.5', '1024', experiment, n_rows, n_queries, n_cols, sel), 'ProgressiveIndexAdaptive (Delta=0.5)'],
# [read_df(files['FullScanCandidateList'], '0.0', '0', experiment, n_rows, n_queries, n_cols, sel), 'Full Scan Candidate List'],
],
    read_df(files['FullScanCandidateList'], '0.0', '0', experiment, n_rows, n_queries, n_cols, sel)
)

In [23]:
# to calculate selectivity in %
# df = read_df('query0', files['CrackingKDTree'], partition='1024')
# df['scan_overhead_after_adapt']/df['tuples_scanned'] * 100

In [55]:
read_df(files['CrackingKDTree'], '0.0', '1024', experiment, n_rows, n_queries, n_cols, sel)

,index,adaptation_time,index_search_time,initialization_time,max_height,memory_footprint,min_height,number_of_nodes,partitions_scanned,partitions_skipped,scan_overhead_after_adapt,scan_overhead_before_adapt,scan_time,tuples_scanned,repetition,query_time,query_time_cumsum
0,0.0,0.259312,0.000005,0.212354,8.0,576.0,8.0,8.0,1.0,0.0,inf,inf,0.000076,39164.0,0.0,0.471747,0.471747
1,1.0,0.000001,0.000000,0.000000,8.0,576.0,8.0,8.0,1.0,0.0,inf,inf,0.000074,39164.0,0.0,0.000075,0.471822
2,2.0,0.000001,0.000001,0.000000,8.0,576.0,8.0,8.0,1.0,0.0,inf,inf,0.000074,39164.0,0.0,0.000076,0.471898
3,3.0,0.000000,0.000001,0.000000,8.0,576.0,8.0,8.0,1.0,0.0,inf,inf,0.000073,39164.0,0.0,0.000074,0.471972
4,4.0,0.000001,0.000000,0.000000,8.0,576.0,8.0,8.0,1.0,0.0,inf,inf,0.000073,39164.0,0.0,0.000074,0.472046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995.0,0.012662,0.003473,0.000000,835.0,1949688.0,3.0,27079.0,25709.0,0.0,5.954722,6.688513,0.082070,3903231.0,0.0,0.098205,8.199119
2996,2996.0,0.017113,0.003685,0.000000,835.0,1992096.0,3.0,27668.0,26196.0,0.0,5.557354,6.486425,0.084733,4047543.0,0.0,0.105531,8.304650
2997,2997.0,0.022752,0.003739,0.000000,835.0,2035512.0,3.0,28271.0,26626.0,0.0,5.182466,6.325896,0.088172,4193781.0,0.0,0.114663,8.419313
2998,2998.0,0.028990,0.003793,0.000000,835.0,2068200.0,3.0,28725.0,26930.0,0.0,4.825938,6.169851,0.090863,4340333.0,0.0,0.123646,8.542959


In [54]:
np.var([2, 2, 2])

0.0

In [11]:
df = pd.DataFrame({'Workloads': ['1','2','3'], 'b': [2, 2, 2]})
print(df.to_latex(index=False))

\begin{tabular}{lr}
\toprule
Workloads &  b \\
\midrule
        1 &  2 \\
        2 &  2 \\
        3 &  2 \\
\bottomrule
\end{tabular}



# Big Latex Table

In [4]:
experiments = [
    'uniform',
    'skewed',
    'sequential',
    'periodic',
    'zoom_in',
    'sequential_zoom_in',
    'alternating_zoom_in',
]
algorithms = [
    'AverageKDTree', 'CrackingKDTree', 'Quasii', 'ProgressiveIndex', 'ProgressiveIndexCostModel'
]

abbreviations = [
    'AKD', 'CKD', 'Q', 'PI', 'PI_CM'
]

deltas = [
    '0.0', '0.0', '0.0', '0.1', '0.1'
]

# 'AverageKDTree': 'average_kd_tree',
#     'MedianKDTree': 'median_kd_tree',
#     'CrackingKDTree': 'cracking_kd_tree',
#     'CrackingKDTreePerDimension': 'cracking_kd_tree_pd',
#     'Quasii': 'quasii',
#     'FullScanBitVector':'full_scan_bv',
#     'FullScanCandidateList': 'full_scan',
#     'ProgressiveIndex': 'progressive_index',
#     'ProgressiveIndexAdaptive': 'progressive_index_adaptive'

n_rows = 10000000
n_queries = 3000
n_cols = [2, 4]
sel = 0.01

data = {'Workload (# columns)': []}
for n_col in n_cols:
    for alg, abbr, delta in zip(algorithms, abbreviations, deltas):
        for exp in experiments:
            if abbr not in data:
                data[abbr] = []
            data[abbr].append(
                read_df(files[alg], delta, '1024', exp, n_rows, n_queries, n_col, sel)['query_time'].sum()
            )
    for exp in experiments:
        data['Workload (# columns)'].append(f'{exp} ({n_col})')

df = pd.DataFrame(data)
print(df.to_latex(index=False, float_format='%.2F'))
df

\begin{tabular}{lrrrrr}
\toprule
    Workload (\# columns) &   AKD &   CKD &     Q &    PI &  PI\_CM \\
\midrule
             uniform (2) &  3.40 &  3.37 &  7.91 &  4.37 &   3.53 \\
              skewed (2) &  3.35 &  2.73 &  5.83 &  4.44 &   3.60 \\
          sequential (2) &  1.46 &  0.44 &  1.04 &  0.59 &   0.56 \\
            periodic (2) &  1.47 &  0.81 &  1.23 &  1.41 &   1.90 \\
             zoom\_in (2) &  2.24 &  4.43 &  2.01 &  3.53 &   2.52 \\
  sequential\_zoom\_in (2) &  1.50 &  0.53 &  0.96 &  1.72 &   1.54 \\
 alternating\_zoom\_in (2) &  1.48 &  0.29 &  0.68 &  2.44 &   1.75 \\
             uniform (4) & 11.72 & 12.64 & 20.69 & 13.91 &  12.49 \\
              skewed (4) & 11.76 &  9.59 & 16.32 & 15.60 &  14.02 \\
          sequential (4) &  1.67 &  0.57 &  0.84 &  0.21 &   0.18 \\
            periodic (4) &  1.69 &  0.82 &  1.16 &  0.85 &   0.75 \\
             zoom\_in (4) &  4.51 &  9.27 &  3.26 &  8.16 &   6.45 \\
  sequential\_zoom\_in (4) &  1.81 &  0.45 &  0.63 & 

,Workload (# columns),AKD,CKD,Q,PI,PI_CM
0,uniform (2),3.395962,3.372642,7.911270,4.371372,3.530417
1,skewed (2),3.354852,2.733785,5.830466,4.441801,3.598815
2,sequential (2),1.455481,0.443336,1.041608,0.592009,0.555581
3,periodic (2),1.472355,0.813984,1.234782,1.411267,1.899809
4,zoom_in (2),2.239980,4.425794,2.010506,3.534693,2.522689
5,sequential_zoom_in (2),1.498262,0.525698,0.959329,1.721204,1.535452
6,alternating_zoom_in (2),1.483695,0.293032,0.677212,2.435014,1.754946
7,uniform (4),11.720871,12.637652,20.686559,13.909664,12.486005
8,skewed (4),11.764996,9.590545,16.320041,15.596035,14.015461
9,sequential (4),1.671004,0.570411,0.835546,0.214795,0.177939
